In [1]:
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge import Rouge
import torch

In [ ]:
# Vorbereitung der Trainingsdaten (Eingabe und Ziel-Zusammenfassung)

# Tokenizer initialisieren
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Datensatz für die Textzusammenfassung
dataset = [
    {
        'input_text': 'Dies ist ein langer Text, der zusammengefasst werden soll.',
        'target_summary': 'Das ist eine Zusammenfassung.'
    },
    {
        'input_text': 'Ein weiterer Text, der zusammengefasst werden soll.',
        'target_summary': 'Die Zusammenfassung davon ist hier.'
    },
    # Weitere Datensätze hier...
]

# Modell initialisieren
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Trainings- und Validierungsdaten erstellen
train_data = dataset[:int(0.8 * len(dataset))]
valid_data = dataset[int(0.8 * len(dataset)):]

# Trainingsschleife
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(10):
    total_loss = 0
    for example in train_data:
        input_text = example['input_text']
        target_summary = example['target_summary']

        # Tokenisierung der Eingabe und Ziel-Zusammenfassung
        input_ids = tokenizer.encode(input_text, truncation=True, padding='longest', return_tensors='pt').to(device)
        target_ids = tokenizer.encode(target_summary, truncation=True, padding='longest', return_tensors='pt').to(device)

        # Modellvorhersage generieren
        outputs = model(input_ids=input_ids, labels=target_ids)
        loss = outputs.loss
        total_loss += loss.item()

        # Backpropagation und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Durchschnittsverlust für die Epoche berechnen
    avg_loss = total_loss / len(train_data)
    print(f'Epoch {epoch+1}: Average Loss: {avg_loss:.4f}')

    # Evaluation auf Validierungsdaten mit ROUGE-Metrik
    rouge = Rouge()
    total_rouge_score = 0
    for example in valid_data:
        input_text = example['input_text']
        target_summary = example['target_summary']

        input_ids = tokenizer.encode(input_text, truncation=True, padding='longest', return_tensors='pt').to(device)
        predicted_summary = model.generate(input_ids)
        predicted_summary = tokenizer.decode(predicted_summary[0], skip_special_tokens=True)

        scores = rouge.get_scores(predicted_summary, target_summary)
        total_rouge_score += scores[0]['rouge-l']['f']

    avg_rouge_score = total_rouge_score / len(valid_data)
    print(f'Epoch {epoch+1}: Average ROUGE Score: {avg_rouge_score:.4f}')
